In [1]:
import pandas as pd
from docplex.mp.model import Model
import tsplib95

In [2]:
sys.version

'3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]'

In [3]:
problem = tsplib95.load('Data/ts225.tsp')
graph = problem.get_graph()
print(problem.render())

NAME: ts225
COMMENT: 225-city problem (Juenger,Raecke,Tschoecke)
TYPE: TSP
DIMENSION: 225
EDGE_WEIGHT_TYPE: EUC_2D
NODE_COORD_SECTION:
1 4000 4000
2 4000 4500
3 4000 5000
4 4000 5500
5 4000 6000
6 4000 6500
7 4000 7000
8 4000 7500
9 4000 8000
10 4000 8500
11 4000 9000
12 4000 9500
13 4000 10000
14 4000 10500
15 4000 11000
16 4000 11500
17 4000 12000
18 4000 12500
19 4000 13000
20 4000 13500
21 4000 14000
22 4000 14500
23 4000 15000
24 4000 15500
25 4000 16000
26 7000 4000
27 7000 4500
28 7000 5000
29 7000 5500
30 7000 6000
31 7000 6500
32 7000 7000
33 7000 7500
34 7000 8000
35 7000 8500
36 7000 9000
37 7000 9500
38 7000 10000
39 7000 10500
40 7000 11000
41 7000 11500
42 7000 12000
43 7000 12500
44 7000 13000
45 7000 13500
46 7000 14000
47 7000 14500
48 7000 15000
49 7000 15500
50 7000 16000
51 10000 4000
52 10000 4500
53 10000 5000
54 10000 5500
55 10000 6000
56 10000 6500
57 10000 7000
58 10000 7500
59 10000 8000
60 10000 8500
61 10000 9000
62 10000 9500
63 10000 10000
64 10000 10500


## Objective function

# $Min \; Z= \; \displaystyle\sum_{i=1}^n\sum_{j=1,j\neq i}^n d_{ij}  x_{ij} $
## $Subject \;to:$
## $\displaystyle\sum_{j=1, j\neq i}^n x_{ij} = 1, \; \forall \;i \in N$
## $\displaystyle\sum_{i=1, i\neq j}^n x_{ij} = 1, \; \forall \;j \in N$
## $\displaystyle\sum_{i\in Q} \sum_{j\neq i, j\in Q} x_{ij} \le |Q| - 1 \; \forall \; Q \subset \{1,...,n\},\; |Q| \ge2  $

In [4]:
arcos = []
distancia = {}
for i in list(problem.get_edges()):
    if problem.get_weight(*i) != 0:
        arcos.append(i)
        distancia[i] = problem.get_weight(*i)
nodes = list(problem.get_nodes())

# Modelo Exacto

In [5]:
mdl = Model('TSP')
x = mdl.binary_var_dict(arcos, name = 'x')
d = mdl.continuous_var_dict(nodes, name = 'd')

In [6]:
mdl.minimize(mdl.sum(distancia[i] * x[i] for i in arcos))

In [7]:
for node in nodes:
    mdl.add_constraint(mdl.sum(x[(i,j)] for i,j in arcos if i == node)==1,ctname= 'out_%d'%node)

for node in nodes:
    mdl.add_constraint(mdl.sum(x[(i,j)] for i,j in arcos if j == node)==1,ctname= 'in_%d'%node)

for i,j in arcos:
    if j !=1:
        mdl.add_indicator(x[(i,j)],d[i]+1==d[j],name='order_(%d, %d)'%(i,j))

In [10]:
mdl.set_time_limit(152)

In [11]:
solucion = mdl.solve(log_output= True)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
CPXPARAM_TimeLimit                               152
Tried aggregator 2 times.
MIP Presolve modified 24976 coefficients.
Aggregator did 24976 substitutions.
Reduced MIP has 25650 rows, 75825 columns, and 176400 nonzeros.
Reduced MIP has 50400 binaries, 0 generals, 0 SOSs, and 50176 indicators.
Presolve time = 0.33 sec. (3767.45 ticks)
Probing time = 0.70 sec. (79.55 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 25650 rows, 75825 columns, and 176400 nonzeros.
Reduced MIP has 50400 binaries, 0 generals, 0 SOSs, and 50176 indicators.
Presolve time = 0.19 sec. (121.18 ticks)
Probing time = 0.20 sec. (23.58 ticks)
Clique table members: 25426.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solutio